In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [2]:
import json

training_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json'
evaluation_solutions_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json'
evaluation_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json'
sample_submission_path = '/kaggle/input/arc-prize-2024/sample_submission.json'
training_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json'
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#function to load JSON data
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

#load each dataset
training_solutions = load_json_data(training_solutions_path)
evaluation_solutions = load_json_data(evaluation_solutions_path)
evaluation_challenges = load_json_data(evaluation_challenges_path)
sample_submission = load_json_data(sample_submission_path)
training_challenges = load_json_data(training_challenges_path)
test_challenges = load_json_data(test_challenges_path)

In [3]:
#inspect the structure of the data
def inspect_data(data, name):
    print(f"Inspecting {name}:")
    if isinstance(data, list):
        print(f"Number of items: {len(data)}")
        if len(data) > 0:
            print(f"Example item: {data[0]}")
    elif isinstance(data, dict):
        print(f"Number of keys: {len(data.keys())}")
        if len(data.keys()) > 0:
            first_key = list(data.keys())[0]
            print(f"Example item under key '{first_key}': {data[first_key]}")
    else:
        print("Unknown data type")
    print("\n")

inspect_data(training_solutions, "Training Solutions")
inspect_data(evaluation_solutions, "Evaluation Solutions")
inspect_data(evaluation_challenges, "Evaluation Challenges")
inspect_data(sample_submission, "Sample Submission")
inspect_data(training_challenges, "Training Challenges")
inspect_data(test_challenges, "Test Challenges")

Inspecting Training Solutions:
Number of keys: 400
Example item under key '007bbfb7': [[[7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 0, 7, 0, 0, 0, 7, 0, 7], [7, 7, 0, 0, 0, 0, 7, 7, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 0, 7, 7, 0, 7, 0, 0, 0], [7, 7, 0, 7, 7, 0, 0, 0, 0]]]


Inspecting Evaluation Solutions:
Number of keys: 400
Example item under key '00576224': [[[3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8], [2, 3, 2, 3, 2, 3], [8, 7, 8, 7, 8, 7], [3, 2, 3, 2, 3, 2], [7, 8, 7, 8, 7, 8]]]


Inspecting Evaluation Challenges:
Number of keys: 400
Example item under key '00576224': {'test': [{'input': [[3, 2], [7, 8]]}], 'train': [{'input': [[8, 6], [6, 4]], 'output': [[8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4], [6, 8, 6, 8, 6, 8], [4, 6, 4, 6, 4, 6], [8, 6, 8, 6, 8, 6], [6, 4, 6, 4, 6, 4]]}, {'input': [[7, 9], [4, 3]], 'output': [[7, 9, 7, 9, 7, 9], [4, 3, 4, 3, 4, 3], [9, 7, 9, 7, 9, 7], [3, 4, 3, 4, 3, 4], [7, 9, 7, 9, 7, 

In [4]:
def get_max_grid_size(challenges, solutions):
    max_input_height, max_input_width = 0, 0
    max_output_height, max_output_width = 0, 0
    
    for key in challenges.keys():
        challenge = challenges[key]
        for example in challenge['train']:
            input_grid = example['input']
            output_grid = example['output']
            max_input_height = max(max_input_height, len(input_grid))
            max_input_width = max(max_input_width, len(input_grid[0]))
            max_output_height = max(max_output_height, len(output_grid))
            max_output_width = max(max_output_width, len(output_grid[0]))
        for test_case in challenge['test']:
            test_input = test_case['input']
            max_input_height = max(max_input_height, len(test_input))
            max_input_width = max(max_input_width, len(test_input[0]))
            #assuming test_output size can be derived similarly

    return max_input_height, max_input_width, max_output_height, max_output_width

max_input_height, max_input_width, max_output_height, max_output_width = get_max_grid_size(training_challenges, training_solutions)
print(f"Max input size: {max_input_height}x{max_input_width}, Max output size: {max_output_height}x{max_output_width}")

Max input size: 30x30, Max output size: 30x30


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ARCDataset(Dataset):
    def __init__(self, challenges, solutions, max_size, transform=None):
        self.data = []
        self.max_size = max_size
        self.transform = transform
        for key in challenges.keys():
            challenge = challenges[key]
            solution = solutions[key]
            for example in challenge['train']:
                input_grid = example['input']
                output_grid = example['output']
                self.data.append((input_grid, output_grid))
            for test_case in challenge['test']:
                test_input = test_case['input']
                test_output = solution[len(self.data) % len(solution)]
                self.data.append((test_input, test_output))

    def __len__(self):
        return len(self.data)

    def pad_grid(self, grid):
        padded_grid = np.zeros(self.max_size)
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                padded_grid[i][j] = grid[i][j]
        return padded_grid

    def __getitem__(self, idx):
        input_grid, output_grid = self.data[idx]
        input_grid = self.pad_grid(input_grid)
        output_grid = self.pad_grid(output_grid)
        input_grid = torch.tensor(input_grid, dtype=torch.float32).unsqueeze(0)
        output_grid = torch.tensor(output_grid, dtype=torch.float32).unsqueeze(0)
        if self.transform:
            input_grid = self.transform(input_grid)
            output_grid = self.transform(output_grid)
        
        #create dummy text, audio, and context inputs
        text_input = torch.zeros((10,))  #example dummy text input
        audio_input = torch.zeros((20,))  #example dummy audio input
        context_input = torch.zeros((5,))  #example dummy context input
        
        return (input_grid, text_input, audio_input, context_input), output_grid

#define the maximum size
max_size = (30, 30)

#create datasets with padding
transform = transforms.Compose([transforms.Lambda(lambda x: x)]) 
training_dataset = ARCDataset(training_challenges, training_solutions, max_size, transform=transform)
evaluation_dataset = ARCDataset(evaluation_challenges, evaluation_solutions, max_size, transform=transform)

#create dataloaders
training_loader = DataLoader(training_dataset, batch_size=8, shuffle=True)
evaluation_loader = DataLoader(evaluation_dataset, batch_size=8, shuffle=False)

In [6]:
def preprocess_input(x, normalize=False, mean=0.5, std=0.5):
    """
    Preprocesses the input tensor `x` and returns it along with dimensions needed by subsequent modules.
    
    Args:
    - x (torch.Tensor): Input tensor with shape (batch_size, channels, height, width).
    - normalize (bool): Whether to normalize the input tensor.
    - mean (float or list): Mean for normalization (scalar or list for each channel).
    - std (float or list): Standard deviation for normalization (scalar or list for each channel).
    
    Returns:
    - x (torch.Tensor): Preprocessed input tensor.
    - input_channels (int): Number of channels in the input tensor.
    - input_height (int): Height of the input tensor.
    - input_width (int): Width of the input tensor.
    
    Raises:
    - ValueError: If the input tensor `x` does not have the expected number of dimensions or channels.
    """
    #ensure the input tensor has 4 dimensions (batch_size, channels, height, width)
    if x.dim() != 4:
        raise ValueError(f"Expected input tensor with 4 dimensions (batch_size, channels, height, width), got {x.dim()} dimensions instead.")
    
    #check the number of channels
    input_channels = x.size(1)
    
    #ensure that the number of channels is 1 or 3 (grayscale or RGB)
    if input_channels not in [1, 3]:
        raise ValueError(f"Expected input tensor with 1 or 3 channels, got {input_channels} channels instead.")
    
    #extract input dimensions
    input_height = x.size(2)
    input_width = x.size(3)
    
    #normalize the input tensor if required
    if normalize:
        if isinstance(mean, list) and isinstance(std, list):
            if len(mean) != input_channels or len(std) != input_channels:
                raise ValueError("Mean and std lists must have the same length as the number of channels.")
            mean = torch.tensor(mean).view(1, input_channels, 1, 1)
            std = torch.tensor(std).view(1, input_channels, 1, 1)
        else:
            mean = torch.tensor([mean] * input_channels).view(1, input_channels, 1, 1)
            std = torch.tensor([std] * input_channels).view(1, input_channels, 1, 1)
        x = (x - mean) / std
    
    return x, input_channels, input_height, input_width

#example usage
x1 = torch.randn(8, 1, 28, 28)  # Grayscale input tensor
x2 = torch.randn(8, 3, 32, 32)  # RGB input tensor

#without normalization
preprocessed_x1, channels1, height1, width1 = preprocess_input(x1)
print(f"Preprocessed tensor shape: {preprocessed_x1.shape}")
print(f"Channels: {channels1}, Height: {height1}, Width: {width1}")

#with normalization
preprocessed_x2, channels2, height2, width2 = preprocess_input(x2, normalize=True, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
print(f"Preprocessed tensor shape: {preprocessed_x2.shape}")
print(f"Channels: {channels2}, Height: {height2}, Width: {width2}")

Preprocessed tensor shape: torch.Size([8, 1, 28, 28])
Channels: 1, Height: 28, Width: 28
Preprocessed tensor shape: torch.Size([8, 3, 32, 32])
Channels: 3, Height: 32, Width: 32


In [7]:
#inspect the structure of a batch from the DataLoader
for batch in training_loader:
    print(type(batch))
    print(len(batch))
    for item in batch:
        print(type(item), item.shape if isinstance(item, torch.Tensor) else len(item))
    break

<class 'list'>
2
<class 'list'> 4
<class 'torch.Tensor'> torch.Size([8, 1, 30, 30])


In [8]:
#define the path to test data
test_challenges_path = '/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json'

#load test data
def load_json_data(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        #print out some information for debugging purposes
        print(f"Successfully loaded data from {file_path}.")
        print(f"Number of keys in the dataset: {len(data.keys())}")
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} does not exist.")
    except json.JSONDecodeError:
        print(f"Error: The file {file_path} is not a valid JSON file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

test_challenges = load_json_data(test_challenges_path)

Successfully loaded data from /kaggle/input/arc-prize-2024/arc-agi_test_challenges.json.
Number of keys in the dataset: 100


In [9]:
import torch.nn.functional as F

#define test dataset
class TestDataset(Dataset):
    def __init__(self, data, target_size=(30, 30), transform=None):
        self.data = data
        self.transform = transform
        self.target_size = target_size
        self.keys = list(data.keys())

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        test_cases = self.data[key]['test']
        #assuming we want to process the first test case
        grid = test_cases[0]['input']
        grid = np.array(grid).astype(np.float32)  #convert to numpy array
        grid = np.expand_dims(grid, axis=0)  #add channel dimension

        #pad grid to the target size
        grid = torch.tensor(grid)  #convert to PyTorch tensor before padding
        grid = self.pad_to_size(grid, self.target_size)

        if self.transform:
            grid = self.transform(grid)
        return grid, key

    def pad_to_size(self, grid, size):
        _, h, w = grid.shape
        target_h, target_w = size
        pad_h = max(0, target_h - h)
        pad_w = max(0, target_w - w)
        padding = (0, pad_w, 0, pad_h)
        #pad using torch tensor
        padded_grid = F.pad(grid, padding, mode='constant', value=0)
        return padded_grid

In [10]:
#define grid dimensions
grid_height = 30
grid_width = 30

#create the dataset and dataloader for the test data
test_dataset = TestDataset(data=test_challenges, target_size=(grid_height, grid_width))
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

#iterate over the DataLoader
for grids, keys in test_loader:
    print(f"Batch of grids shape: {grids.shape}")
    print(f"Keys: {keys}")

Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('007bbfb7', '00d62c1b', '017c7c7b', '025d127b', '045e512c', '0520fde7', '05269061', '05f2a901')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('06df4c85', '08ed6ac7', '09629e4f', '0962bcdd', '0a938d79', '0b148d64', '0ca9ddb6', '0d3d703e')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('0dfd9992', '0e206a2e', '10fcaaa3', '11852cab', '1190e5a7', '137eaa0f', '150deff5', '178fcbfb')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('1a07d186', '1b2d62fb', '1b60fb0c', '1bfc4729', '1c786137', '1caeab9d', '1cf80156', '1e0a9b12')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('1e32b0e9', '1f0c79e5', '1f642eb9', '1f85a75f', '1f876c06', '1fad071e', '2013d3e2', '2204b7a8')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('22168020', '22233c11', '2281f1f4', '228f6490', '22eb0ac0', '234bbc79', '23581191', '239be575')
Batch of grids shape: torch.Size([8, 1, 30, 30])
Keys: ('23b5c85d', '253bf280', '25d487e

In [11]:
import torch.nn as nn
import torch.optim as optim

#define the Ethical Guidelines class
class EthicalGuidelines:
    def __init__(self, guidelines):
        self.guidelines = guidelines

    def apply_guidelines(self, predictions):
        if self.guidelines['cultural_heritage']:
            predictions = predictions * 1.05
        if self.guidelines['honor_issue'] > 0.1:
            honor_adjustment = self.guidelines['honor_issue']
            predictions = predictions * (1 + honor_adjustment)
        if self.guidelines['nature_related'] > 0.2:
            nature_adjustment = self.guidelines['nature_related']
            predictions = predictions * (1 + nature_adjustment)
        if self.guidelines['family_involved']:
            predictions = predictions * 1.03 
        if self.guidelines['bravery_needed']:
            predictions = predictions * 1.02
        if self.guidelines['self_reliance_needed']:
            predictions = predictions * 1.01
        if self.guidelines['guest_involved']:
            predictions = predictions * 1.04 
        if self.guidelines['consensus_needed']:
            predictions = predictions * 1.02 

        #normalize predictions to ensure they are within a valid range
        predictions = torch.clamp(predictions, min=0.0, max=1.0)

        return predictions

#define the Cognitive Architecture model
class CognitiveArchitecture(nn.Module):
    def __init__(self):
        super(CognitiveArchitecture, self).__init__()
        self.fc = nn.Linear(30 * 30, 100)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return F.relu(self.fc(x))

#define the Meta Learning model
class MetaLearning(nn.Module):
    def __init__(self):
        super(MetaLearning, self).__init__()
        self.fc = nn.Linear(10, 100)  

    def forward(self, x):
        return F.relu(self.fc(x))

#define the Embodied Cognition model
class EmbodiedCognition(nn.Module):
    def __init__(self):
        super(EmbodiedCognition, self).__init__()
        self.fc = nn.Linear(20, 100) 

    def forward(self, x):
        return F.relu(self.fc(x))

#define the Bias Detection class
class BiasDetection:
    def __init__(self):
        self.bias_threshold = 0.1
    
    def check_bias(self, predictions):
        bias_detected = torch.any(predictions > self.bias_threshold)
        return bias_detected

#define the Self Improvement System
class SelfImprovementSystem:
    def __init__(self, model, ethical_guidelines):
        self.model = model
        self.ethical_guidelines = ethical_guidelines
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        self.loss_function = nn.CrossEntropyLoss()

    def detect_and_correct_errors(self, test_data, test_labels):
        self.collect_data(test_data, test_labels)
        self.retrain()

    def collect_data(self, test_data, test_labels):
        self.train_data = test_data
        self.train_labels = test_labels

    def retrain(self):
        self.model.train()
        inputs = self.train_data
        targets = self.train_labels

        image_input, text_input, audio_input = inputs

        predictions = self.model(image_input, text_input, audio_input)
        loss = self.loss_function(predictions, targets)

        print("Loss before backward:", loss.item())

        self.optimizer.zero_grad()
        loss.backward()

        for name, param in self.model.named_parameters():
            if param.requires_grad:
                print(f"Parameter {name} requires grad: {param.requires_grad}")
                if param.grad is not None:
                    print(f"Parameter {name} has gradient.")
                else:
                    print(f"Parameter {name} has no gradient.")

        self.optimizer.step()

        self.train_data = None
        self.train_labels = None

#define the Yambi ("Welcome" in Kikongo) model
class Yambi(nn.Module):
    def __init__(self, ethical_guidelines):
        super(Yambi, self).__init__()
        self.cognitive_architecture = CognitiveArchitecture()
        self.meta_learning = MetaLearning()
        self.embodied_cognition = EmbodiedCognition()
        self.final_fc = nn.Linear(300, 10)
        self.ethical_guidelines = ethical_guidelines
        self.output_layer = nn.Linear(300, 10)
        self.self_improvement = SelfImprovementSystem(self, ethical_guidelines)
        self.bias_detection = BiasDetection()

    def forward(self, image_input, text_input, audio_input):
        cognitive_output = self.cognitive_architecture(image_input)
        meta_output = self.meta_learning(text_input)
        embodied_output = self.embodied_cognition(audio_input)

        combined_output = torch.cat((cognitive_output, meta_output, embodied_output), dim=1)
        combined_output = F.relu(self.output_layer(combined_output))

        ethical_output = self.ethical_guidelines.apply_guidelines(combined_output)
        final_output = F.softmax(ethical_output, dim=-1)

        return final_output

    def retrain_model(self, test_data, test_labels):
        self.self_improvement.detect_and_correct_errors(test_data, test_labels)

#test Yambi
ethical_guidelines = EthicalGuidelines({
    'cultural_heritage': True,
    'honor_issue': 0.1,
    'nature_related': 0.2,
    'family_involved': True,
    'bravery_needed': True,
    'self_reliance_needed': True,
    'guest_involved': True,
    'consensus_needed': True
})

yambi_model = Yambi(ethical_guidelines)
optimizer = optim.Adam(yambi_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

#create test data
test_image_data = torch.randn(8, 1, 30, 30)  
test_text_data = torch.randn(8, 10)
test_audio_data = torch.randn(8, 20) 
test_labels = torch.randint(0, 10, (8,))

#forward pass
output = yambi_model(test_image_data, test_text_data, test_audio_data)
print("Output:", output)

#bias detection
bias_detector = BiasDetection()
bias_detected = bias_detector.check_bias(output)
print("Bias detected:", bias_detected.item())

#retrain the model if needed
yambi_model.retrain_model((test_image_data, test_text_data, test_audio_data), test_labels)

Output: tensor([[0.0887, 0.1006, 0.0852, 0.1053, 0.0806, 0.1314, 0.0929, 0.0806, 0.1543,
         0.0806],
        [0.0985, 0.1073, 0.0917, 0.0909, 0.1123, 0.1276, 0.0909, 0.0987, 0.0909,
         0.0909],
        [0.0946, 0.0989, 0.0920, 0.0846, 0.0868, 0.1119, 0.0846, 0.1352, 0.1268,
         0.0846],
        [0.1494, 0.0950, 0.0792, 0.1085, 0.0785, 0.1035, 0.0785, 0.0808, 0.1482,
         0.0785],
        [0.0895, 0.1077, 0.0895, 0.0895, 0.0895, 0.1282, 0.1090, 0.0895, 0.1181,
         0.0895],
        [0.0879, 0.0888, 0.0879, 0.1003, 0.0879, 0.1059, 0.0902, 0.1275, 0.1356,
         0.0879],
        [0.1093, 0.0892, 0.0892, 0.0892, 0.0934, 0.1071, 0.1077, 0.1098, 0.1160,
         0.0892],
        [0.0988, 0.1012, 0.0914, 0.0921, 0.0989, 0.1037, 0.0979, 0.1013, 0.1234,
         0.0914]], grad_fn=<SoftmaxBackward0>)
Bias detected: True
Loss before backward: 2.3055713176727295
Parameter cognitive_architecture.fc.weight requires grad: True
Parameter cognitive_architecture.fc.weight has 

In [12]:
def inspect_dataloader(dataloader):
    for batch_idx, batch in enumerate(dataloader):
        if isinstance(batch, (tuple, list)):
            inputs, labels = batch
            print(f"Batch {batch_idx+1}:")
            print(f"  Inputs (batch size {len(inputs)}):")
            if isinstance(inputs, torch.Tensor):
                print(f"    Type: {type(inputs)}")
                print(f"    Shape: {inputs.shape}")
                print(f"    Sample Data: {inputs[:2]}")  #print first 2 samples
            else:
                print(f"    Type: {type(inputs)} (not a tensor)")
                print(f"    Sample Data: {inputs[:2]}")  #print first 2 samples if not a tensor

            print(f"  Labels (batch size {len(labels)}):")
            if isinstance(labels, torch.Tensor):
                print(f"    Type: {type(labels)}")
                print(f"    Shape: {labels.shape}")
                print(f"    Sample Labels: {labels[:2]}")  #print first 2 labels
            else:
                print(f"    Type: {type(labels)} (not a tensor)")
                print(f"    Sample Labels: {labels[:2]}")  #print first 2 labels if not a tensor
        else:
            print(f"Batch {batch_idx+1} is not a tuple or list.")
        break 

inspect_dataloader(evaluation_loader)

Batch 1:
  Inputs (batch size 4):
    Type: <class 'list'> (not a tensor)
    Sample Data: [tensor([[[[8., 6., 0.,  ..., 0., 0., 0.],
          [6., 4., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[7., 9., 0.,  ..., 0., 0., 0.],
          [4., 3., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[3., 2., 0.,  ..., 0., 0., 0.],
          [7., 8., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 8., 0.,  ..., 0., 0., 0